# 1. Testy poprawności wymiarów (SILVER)

## 1.1. Test: brak duplikatów kluczy w wymiarach

In [0]:
SELECT airline_key, COUNT(*)
FROM project_silver.DimAirlines
GROUP BY airline_key
HAVING COUNT(*) > 1;

airline_key,COUNT(*)


In [0]:
SELECT airport_key, COUNT(*)
FROM project_silver.DimAirports
GROUP BY airport_key
HAVING COUNT(*) > 1;

airport_key,COUNT(*)


In [0]:
SELECT date_key, COUNT(*)
FROM workspace.project_silver.DimDate
GROUP BY date_key
HAVING COUNT(*) > 1;

date_key,COUNT(*)


## 1.2. Test poprawności SCD2 (czy istnieje dokładnie 1 rekord bieżący)

In [0]:
SELECT iata_code, COUNT(*) AS current_count
FROM workspace.project_silver.DimAirlines
WHERE is_current = true
GROUP BY iata_code
HAVING COUNT(*) != 1;

iata_code,current_count


# 2. Testy integracji wymiarów z fact (GOLD)

## 2.1. Test spójności kluczy obcych

In [0]:
SELECT DISTINCT airline_key
FROM workspace.project_gold.FactTable
WHERE airline_key NOT IN (
  SELECT airline_key FROM workspace.project_silver.DimAirlines
);

airline_key


In [0]:
SELECT DISTINCT origin_airport_key
FROM workspace.project_gold.FactTable
WHERE origin_airport_key NOT IN (
  SELECT airport_key FROM workspace.project_silver.DimAirports
);

origin_airport_key


In [0]:
SELECT DISTINCT destination_airport_key
FROM workspace.project_gold.FactTable
WHERE destination_airport_key NOT IN (
  SELECT airport_key FROM workspace.project_silver.DimAirports
);

destination_airport_key


In [0]:
SELECT DISTINCT date_key
FROM workspace.project_gold.FactTable
WHERE date_key NOT IN (
  SELECT date_key FROM workspace.project_silver.DimDate
);

date_key


## 2.2. Test: czy FactTable nie zawiera nulli w kluczach obcych

In [0]:
SELECT *
FROM workspace.project_gold.FactTable
WHERE airline_key IS NULL
   OR origin_airport_key IS NULL
   OR destination_airport_key IS NULL
   OR date_key IS NULL;

flight_key,date_key,airline_key,origin_airport_key,destination_airport_key,flight_number,tail_number,scheduled_departure,departure_time,departure_delay,taxi_out,wheels_off,scheduled_time,elapsed_time,air_time,distance,wheels_on,taxi_in,scheduled_arrival,arrival_time,arrival_delay,diverted,cancelled,cancellation_reason,air_system_delay,security_delay,airline_delay,late_aircraft_delay,weather_delay,load_timestamp


# 3. Testy jakości danych (miary w fact)

## 3.1. Test: distance musi być > 0

In [0]:
SELECT *
FROM workspace.project_gold.FactTable
WHERE distance <= 0;

flight_key,date_key,airline_key,origin_airport_key,destination_airport_key,flight_number,tail_number,scheduled_departure,departure_time,departure_delay,taxi_out,wheels_off,scheduled_time,elapsed_time,air_time,distance,wheels_on,taxi_in,scheduled_arrival,arrival_time,arrival_delay,diverted,cancelled,cancellation_reason,air_system_delay,security_delay,airline_delay,late_aircraft_delay,weather_delay,load_timestamp


# 4. Testy kompletności przetwarzania (ETL end-to-end)

## 4.1. Test: liczba rekordów BRONZE = liczbie rekordów załadowanych do GOLD

In [0]:
SELECT
  (SELECT COUNT(*) FROM workspace.project_bronze.flights) AS bronze_count,
  (SELECT COUNT(*) FROM workspace.project_gold.FactTable) AS gold_count;

bronze_count,gold_count
5231130,21026304


## 4.2. Test: czy wszystkie rekordy BRONZE znalazły się w GOLD (po kluczu naturalnym)

In [0]:
SELECT b.year, b.month, b.day, f.flight_key
FROM workspace.project_bronze.flights b
LEFT JOIN workspace.project_silver.dimdate d
  ON b.year = d.year AND b.month = d.month AND b.day = d.day
JOIN workspace.project_gold.FactTable f
  ON d.date_key = f.date_key
WHERE f.flight_key IS NULL;

year,month,day,flight_key


# 5. Testy agregacji (GOLD → raporty)

## 5.1. Test: dzienne agregacje powinny zawierać unikalny date_key

In [0]:
SELECT date_key, COUNT(*)
FROM workspace.project_gold.Agg_DailyDelays
GROUP BY date_key
HAVING COUNT(*) > 1;

date_key,COUNT(*)


## 5.2. Test: suma opóźnień z agregacji = suma w fact

In [0]:
SELECT
  (SELECT SUM(arrival_delay) FROM workspace.project_gold.FactTable) AS fact_sum,
  (SELECT SUM(avg_arrival_delay * total_flights) FROM workspace.project_gold.Agg_DailyDelays) AS agg_sum;

fact_sum,agg_sum
102235176,7.6676382E7


# 6. Testy funkcjonalne

## 6.1. Czy raport punktualności działa?

In [0]:
SELECT airline, AVG(arrival_delay) AS avg_delay
FROM workspace.project_gold.vw_flight_delays
GROUP BY airline;

airline,avg_delay
Frontier Airlines Inc.,13.698980988593156
US Airways Inc.,3.6849026854913847
Southwest Airlines Co.,4.817739698000323
Atlantic Southeast Airlines,6.908199650432677
Delta Air Lines Inc.,0.6790874094913935
Alaska Airlines Inc.,-0.7901145869924928
American Airlines Inc.,3.916540153129588
JetBlue Airways,6.914586971441348
Spirit Air Lines,15.134723024614773
Skywest Airlines Inc.,6.188590506413422


# 7. Testy dodawania nowych rekordów

## 7.1 Dodanie nowych rekordów w DimAirlines

In [0]:
INSERT INTO project_bronze.airlines (iata_code, airline) VALUES
('AA', 'Update'),
('PL', 'PLL LOT');

num_affected_rows,num_inserted_rows
2,2


In [0]:
MERGE INTO project_silver.DimAirlines AS d
USING (
    SELECT
        iata_code,
        airline
    FROM project_bronze.airlines
) AS a
ON d.iata_code = a.iata_code AND d.is_current = true
 
WHEN MATCHED AND (
  d.airline <> a.airline
) THEN
  UPDATE SET
    d.valid_to   = current_timestamp(),
    d.is_current = false
 
WHEN NOT MATCHED THEN
  INSERT (
    iata_code, airline, valid_from, valid_to, is_current
  )
  VALUES (
    a.iata_code, a.airline, current_timestamp(), DATE '9999-12-31', true
  );

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
1,1,0,0


In [0]:
SELECT * FROM project_silver.DimAirlines

airline_key,iata_code,airline,valid_from,valid_to,is_current
2,AA,American Airlines Inc.,2025-12-04,2025-12-04,false
15,PL,PLL LOT,2025-12-04,9999-12-31,true
16,AA,American Airlines Inc.,2026-02-21,2026-02-21,false
1,UA,United Air Lines Inc.,2025-12-04,9999-12-31,true
3,US,US Airways Inc.,2025-12-04,9999-12-31,true
4,F9,Frontier Airlines Inc.,2025-12-04,9999-12-31,true
5,B6,JetBlue Airways,2025-12-04,9999-12-31,true
6,OO,Skywest Airlines Inc.,2025-12-04,9999-12-31,true
7,AS,Alaska Airlines Inc.,2025-12-04,9999-12-31,true
8,NK,Spirit Air Lines,2025-12-04,9999-12-31,true


## 7.2 Dodanie nowych rekordów w DimAirports

In [0]:
INSERT INTO project_bronze.airports (iata_code, airport, city, state, country, latitude, longitude) VALUES
('KRK', 'Kraków John Paul II International Airport', 'Kraków', 'małopolskie', 'Poland', 50.0777, 19.7848);

INSERT INTO project_bronze.airports (iata_code, airport, city, state, country, latitude, longitude) VALUES
('ABE', 'Update name', 'Allentown', 'PA', 'USA', 40.65235900878906, -75.44039916992188);

num_affected_rows,num_inserted_rows
1,1


In [0]:
MERGE INTO project_silver.DimAirports AS d
USING (
    SELECT
        iata_code,
        airport,
        city,
        state,
        country,
        latitude,
        longitude
    FROM project_bronze.airports
) AS a
ON d.iata_code = a.iata_code AND d.is_current = true
 
WHEN MATCHED AND (
  d.airport <> a.airport
  OR d.city <> a.city
  OR d.state <> a.state
  OR d.country <> a.country
  OR d.latitude <> a.latitude
  OR d.longitude <> a.longitude

) THEN
  UPDATE SET
    d.valid_to   = current_date(),
    d.is_current = false
 
WHEN NOT MATCHED THEN
  INSERT (
    iata_code, airport, city, state, country, latitude, longitude, valid_from, valid_to, is_current
  )
  VALUES (
    a.iata_code, a.airport, a.city, a.state, a.country, a.latitude, a.longitude, current_date(), DATE '9999-12-31', true
  );


num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
1,1,0,0


In [0]:
SELECT * FROM project_silver.DimAirports LIMIT 20;

airport_key,iata_code,airport,city,state,country,latitude,longitude,valid_from,valid_to,is_current
2,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.6819,2025-12-04,9999-12-31,true
3,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919,2025-12-04,9999-12-31,true
4,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183,2025-12-04,9999-12-31,true
5,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447,2025-12-04,9999-12-31,true
6,ACK,Nantucket Memorial Airport,Nantucket,MA,USA,41.25305,-70.06018,2025-12-04,9999-12-31,true
7,ACT,Waco Regional Airport,Waco,TX,USA,31.61129,-97.23052,2025-12-04,9999-12-31,true
8,ACV,Arcata Airport,Arcata/Eureka,CA,USA,40.97812,-124.10862,2025-12-04,9999-12-31,true
9,ACY,Atlantic City International Airport,Atlantic City,NJ,USA,39.45758,-74.57717,2025-12-04,9999-12-31,true
10,ADK,Adak Airport,Adak,AK,USA,51.87796,-176.64603,2025-12-04,9999-12-31,true
11,ADQ,Kodiak Airport,Kodiak,AK,USA,57.74997,-152.49387,2025-12-04,9999-12-31,true
